# Figure 4

Simple models for radar power loss in glacial ice. Testing ideas for measured power suppression at active subglacial lakes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Bessel function for surface scattering
from scipy.special import i0
# for Mie scattering
import miepython

In [ ]:
### Surface Scattering ###

epsr_ice = 3.15   # ice permittivity
nice = np.sqrt(epsr_ice) # refractive index
fc = 300e6  # center frequency
vice = 3e8/nice # ice velocity
λ = vice/fc # wavelength

S = np.linspace(0,10.,100)  # RMS surface roughness
g = 4.*np.pi*S/λ   # RMS phase variation
ρ_bed = np.exp(-g**2.)*i0(g**2./2.)**2. # power losses at the bed

g = 4.*np.pi*(S/λ)*(np.sqrt(epsr_ice)-1.) # RMS phase with refraction
ρ_surf = np.exp(-g**2.)*i0(g**2./2.)**2. # power losses at the surface

In [ ]:
### Volume Scattering ###

nwater = np.sqrt(80)  # refractive index of water
nsediment = np.sqrt(5.) # refractive index of sediment like granite

r = np.linspace(0.00001,.25,100) # particle radius
x = 2.*np.pi*r/λ # 

# Water Particles #
m = nwater/nice # relative refractive index
qext, q_water, qback, g = miepython.mie(m, x) # Mie scattering model
ϕ = np.array([.1,.3]) # porosity
τ = (ϕ[0]/(4./3.*r), ϕ[1]/(4./3.*r))*q_water # power losses for each porosity
L_water = 10.*np.log10(np.exp(2.*τ)) # log power losses

# Sedimetn Particles #
m = nsediment/nice # relative refractive index
qext, q_sediment, qback, g = miepython.mie(m, x) # Mie scattering model
τ = (ϕ[0]/(4./3.*r), ϕ[1]/(4./3.*r))*q_sediment # power losses for each porosity
L_sediment = 10.*np.log10(np.exp(2.*τ)) # log power losses

In [ ]:
### Thermal Attenuation ###

T0 = 273.5 # T0 for Kelvin conversion
Ts = T0 + np.linspace(-30,0,1000) # range of temperatures

# MacGregor 2007
σ0 = 7.2 # conductivity
E0 = .55 # activation energy
k = 8.6173303e-5 # Boltzmann constant
Tr = 251. # reference temperature
σ = σ0*np.exp(E0/k*(1./Tr-1./Ts)) # Arrhenius conductivity model

# Winebrenner (2003)
N_temp = 0.912*σ # Attenuation rate

In [ ]:
### Water Attenuation (Geldsetzer et al., 2009; Kendrick et al., 2018) ###

ϕs = np.linspace(0.0,0.3,100) # range of porosities
y = 1.67 # degree of conductivity dependence on porosity
ϕc = 0.0 # connectivity between powers

# Low conductivity water
σw1 = 530 # source water conductivity (low)
σws1 = σw1*(ϕs)**y + σ0*(1.-ϕs)*np.exp(E0/k*(1./Tr-1./T0))
N_high = 0.912*σws1 # Attenuation rate

# High conductivity water
σw2 = 72000 # source water conductivity (high)
σws2 = σw2*(ϕs)**y + σ0*(1.-ϕs)*np.exp(E0/k*(1./Tr-1./T0))
N_low = 0.912*σws2 # Attenuation rate

In [ ]:
fig = plt.figure(figsize=(12,3))
gs = GridSpec(2,4)

### Surface Scattering ###

ax1 = plt.subplot(gs[:,0])
plt.plot(S,-10.*np.log10(ρ_bed),'k')
plt.plot(S,-10.*np.log10(ρ_surf),'k:')
plt.ylim(0,40)
plt.xlim(0,1.)
plt.ylabel('Power Loss (dB)')
plt.xlabel('RMS Height (m)')


### Volume Scattering ###

ax2 = plt.subplot(gs[0,1])
plt.plot(r,L_water[0],'steelblue',ls='--')
plt.plot(r,L_water[1],'steelblue',ls=':')
plt.plot(r,L_sediment[0],'k',ls='--')
plt.plot(r,L_sediment[1],'k',ls=':')
plt.xlim(0,.1)
plt.ylim(2.5,100)
plt.ylabel('(dB/m)')

ax3 = plt.subplot(gs[1,1])
plt.plot(r,L_water[0],'steelblue',ls='--')
plt.plot(r,L_water[1],'steelblue',ls=':')
plt.plot(r,L_sediment[0],'k',ls='--')
plt.plot(r,L_sediment[1],'k',ls=':')
plt.xlim(0,.1)
plt.ylim(0,2.5)
plt.xlabel('Pore Radius (m)')

# hide the spines between ax and ax2
ax2.spines.bottom.set_visible(False)
ax3.spines.top.set_visible(False)
ax2.xaxis.tick_top()
ax2.tick_params(labeltop=False)  # don't put tick labels at the top
ax3.xaxis.tick_bottom()

d = .5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax2.plot([0, 1], [0, 0], transform=ax2.transAxes, **kwargs)
ax3.plot([0, 1], [1, 1], transform=ax3.transAxes, **kwargs)


### Thermal Attenuation ###

ax4 = plt.subplot(gs[:,2])
ax4.tick_params(labelleft=False)
plt.plot(Ts-T0,N_temp/1000.,c='k')
plt.xlim(-30,0)
plt.xticks([-30,-20,-10,0])
ax4.yaxis.tick_right()
plt.xlabel('Ice Temperature ($^\circ$C)')
plt.ylim(-.01,.5)


### Englacial Water Attenuation ###

ax5 = plt.subplot(gs[:,3])
plt.fill_between(ϕs,N_low/1000.,N_high/1000.,color='skyblue')
plt.plot(ϕs,N_low/1000.,'navy')
plt.plot(ϕs,N_high/1000.,c='navy')
plt.xlim(0,.3)
plt.xticks([0,0.1,0.2,0.3])
plt.ylim(-.01,.5)
plt.xlabel('Water Content')
plt.ylabel('Attenuation Rate (dB/m)')
ax5.yaxis.set_label_position("right")
ax5.yaxis.tick_right()

fig.subplots_adjust(hspace=.05)  # adjust space between axes
plt.tight_layout()
plt.savefig('PowerLosses.pdf')